In [1]:
%load_ext autoreload
%autoreload 2

import time
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

# Load CoverType from OpenML

In [2]:
X, y_mc = datasets.fetch_openml(data_id=180, return_X_y=True, as_frame=False)

In [3]:
# make it a binary classification problem for the sake of simplicity
y = y_mc == "Lodgepole_Pine"
xtrain, xtest, ytrain, ytest, ytrain_mc, ytest_mc = train_test_split(X, y, y_mc, test_size=0.2, shuffle=True)

# Train an XGBoost model

In [4]:
import veritas
import xgboost as xgb
import tqdm
from sklearn.metrics import accuracy_score, roc_auc_score

In [5]:
params = {
    "n_estimators": 100,
    "eval_metric": "auc",
    "seed": 197,
    "max_depth": 10,
    "learning_rate": 0.2
}
model = xgb.XGBClassifier(**params)

t = time.time()
model.fit(X, y)
print(f"XGB trained in {time.time()-t} seconds")

XGB trained in 7.92949366569519 seconds


In [6]:
ytrain_pred = model.predict(xtrain)
ytest_pred = model.predict(xtest)
acc_train = accuracy_score(ytrain, ytrain_pred)
acc_test = accuracy_score(ytest, ytest_pred)
auc_train = roc_auc_score(ytrain, ytrain_pred)
auc_test = roc_auc_score(ytest, ytest_pred)

print(f"Train acc: {acc_train:.3f}, test acc: {acc_test:.3f} wrt true labels")
print(f"Train auc: {auc_train:.3f}, test auc: {auc_test:.3f} wrt true labels")

Train acc: 0.939, test acc: 0.939 wrt true labels
Train auc: 0.940, test auc: 0.940 wrt true labels


# Generate adversarial examples

We only allow changes to the first 10 numerical attribute values. The remaining 44 attribute values are binary.

In [7]:
# FROM CLASS 0 -> CLASS 1
number_of_adv_examples = 10

rng = np.random.default_rng(seed=19656)
xtest0 = xtest[ytest==0, :]
subset = xtest0[rng.choice(range(xtest0.shape[0]), number_of_adv_examples), :]

eps = 0.2

In [8]:
feat2id = lambda s: int(s[1:])
at = veritas.addtree_from_xgb_model(model, feat2id)
Xstd = xtrain.std(axis=0)
Xstd[10:] = 0.0 # allow no change in binary attributes

In [9]:
adv_examples = []
for i in tqdm.tqdm(range(subset.shape[0])):
    base_example = subset[i, :]
    
    # allow each attribute to vary by eps*stddev of attribute to either side
    box = [veritas.Domain(x-eps*s, x+eps*s) for x, s in zip(base_example, Xstd)]
    
    s = veritas.Search.max_output(at)
    s.prune(box)

    # continue the search for at most 1 second
    # stop early when the optimal solution is found
    stop_reason = s.step_for(1.0, 1000)
    
    if s.num_solutions() > 0:
        sol = s.get_solution(0)
        adv_example = veritas.get_closest_example(sol, base_example)

        res = {"i": i, "adv_example": adv_example, "base_example": base_example}

        res["base_ypred_at"] = at.predict_proba(np.atleast_2d(base_example))[0]
        res["base_ypred"] = model.predict_proba(np.atleast_2d(base_example))[0,1]
        res["adv_ypred"] = model.predict_proba(np.atleast_2d(adv_example))[0,1]
        res["adv_ypred_at"] = at.predict_proba(np.atleast_2d(adv_example))[0]
        res["optimal"] = s.is_optimal()

        adv_examples.append(res)
    else:
        print("no adversarial examples found for", i)

100%|██████████| 10/10 [00:00<00:00, 29.81it/s]


In [10]:
print("i  {:15}{:15}{:>8}{:>10}".format("p(y=1|normal)", "p(y=1|adv)", "success?", "optimal?"))
for i, res in enumerate(adv_examples):
    success = "y" if res["adv_ypred"]>0.5 else "-"
    optimal = "y" if res["optimal"] else "-"
    base = res["base_ypred"]
    adv = res["adv_ypred"]
    print("{:<3}{:>13}  {:>10}     {:>8}{:>10}".format(i, f'{base*100:4.1f}%', f'{adv*100:4.1f}%', success, optimal))

i  p(y=1|normal)  p(y=1|adv)     success?  optimal?
0           0.2%        6.9%            -         y
1           4.3%       60.7%            y         y
2          10.2%       69.5%            y         y
3           5.2%       60.6%            y         y
4          11.0%       59.3%            y         y
5          51.3%       98.3%            y         y
6           0.4%       22.4%            -         y
7           0.2%       12.2%            -         y
8           3.2%       84.4%            y         y
9          13.6%       47.5%            -         y


**Which attributes differ?**

> Name / Data Type / Measurement / Description
> --------------------------------------------
> Elevation / quantitative /meters / Elevation in meters  
> Aspect / quantitative / azimuth / Aspect in degrees azimuth  
> Slope / quantitative / degrees / Slope in degrees  
> Horizontal_Distance_To_Hydrology / quantitative / meters / Horz Dist to nearest surface water features  
> Vertical_Distance_To_Hydrology / quantitative / meters / Vert Dist to nearest surface water features  
> Horizontal_Distance_To_Roadways / quantitative / meters / Horz Dist to nearest roadway   
> Hillshade_9am / quantitative / 0 to 255 index / Hillshade index at 9am, summer solstice   
> Hillshade_Noon / quantitative / 0 to 255 index / Hillshade index at noon, summer solstice   
> Hillshade_3pm / quantitative / 0 to 255 index / Hillshade index at 3pm, summer solstice   
> Horizontal_Distance_To_Fire_Points / quantitative / meters / Horz Dist to nearest wildfire ignition points   
> Wilderness_Area (4 binary columns) / qualitative / 0 (absence) or 1 (presence) / Wilderness area designation   
> Soil_Type (40 binary columns) / qualitative / 0 (absence) or 1 (presence) / Soil Type designation   
> Cover_Type (7 types) / integer / 1 to 7 / Forest Cover Type designation 

In [11]:
import pprint
attribute_names = ["elevation", "aspect", "slope", "hoz_dist_hydro", "ver_dist_hydro", "hoz_dist_road", "shade_9am", "shade_noon", "shade_3am", "hoz_dist_fire"]# + [f"wilderness{k}" for k in range(4)] + [f"soil{k}" for k in range(40)]

for i, adv in enumerate(adv_examples):
    base_ex, adv_ex = adv["base_example"], adv_examples[i]["adv_example"]
    print(f'i={i} p(y=1) = {adv["base_ypred"]*100:.1f}% -> {adv["adv_ypred"]*100:.1f}%')
    print("    {:15}{:>10}{:>10}{:>10}".format("attribute", "normal", "adv.", "diff."))
    for j, attrname in enumerate(attribute_names):
        b, a = base_ex[j], adv_ex[j]
        print("    {:15}{:>10}{:>10}{:>10}".format(attrname, f'{b:.1f}', f'{a:.1f}', f'{a-b:.1f}'))

i=0 p(y=1) = 0.2% -> 6.9%
    attribute          normal      adv.     diff.
    elevation          3481.0    3429.5     -51.5
    aspect               69.0      67.5      -1.5
    slope                18.0      18.5       0.5
    hoz_dist_hydro       90.0      76.0     -14.0
    ver_dist_hydro        2.0      -4.5      -6.5
    hoz_dist_road      1764.0    1573.5    -190.5
    shade_9am           236.0     234.5      -1.5
    shade_noon          203.0     204.5       1.5
    shade_3am            92.0      92.0       0.0
    hoz_dist_fire      1966.0    2219.5     253.5
i=1 p(y=1) = 4.3% -> 60.7%
    attribute          normal      adv.     diff.
    elevation          3213.0    3159.5     -53.5
    aspect              126.0     132.5       6.5
    slope                23.0      22.5      -0.5
    hoz_dist_hydro      335.0     350.5      15.5
    ver_dist_hydro       59.0      65.5       6.5
    hoz_dist_road      3770.0    3770.0       0.0
    shade_9am           251.0     247.5      -3